<a href="https://colab.research.google.com/github/PedrinHm/Labs_Machine_Learning/blob/main/lab_do_pedrin_regressao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## **Projeto Prático - Regressão Linear (AirBnb)**


---


<br>
UNIRV - UNIVERSIDADE DE RIO VERDE<br>
Faculdade de Engenharia de Software<br><br>


**Aluno: Pedro Henrique Mendes**<br>
pedro.h.mendes@academico.unirv.edu.br<br>
**Prof. Me. Sandro Silva Moreira**<br>
sandro.moreira@unirv.edu.br<br><br>


---



In [65]:
%tensorflow_version 2.x
import tensorflow as tf
print("You are using TensorFlow version", tf.__version__)
if len(tf.config.list_physical_devices('GPU')) > 0:
  print("You have a GPU enabled.")
else:
  print("Enable a GPU before running this notebook.")

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
You are using TensorFlow version 2.15.0
Enable a GPU before running this notebook.


In [66]:
pip install category_encoders

In [67]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from category_encoders.one_hot import OneHotEncoder, OrdinalEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [68]:
# Carregando o arquivo CSV.

url = "http://data.insideairbnb.com/united-states/ny/new-york-city/2024-02-06/data/listings.csv.gz"

dataset = pd.read_csv(url)

In [69]:
# selecionando um subconjunto específico de colunas.

dataset = dataset[["neighbourhood_cleansed", "property_type", "room_type", "accommodates", "bathrooms", "bedrooms", "beds", "price"]]

In [88]:
# fornece informações detalhadas sobre o DataFrame.

dataset.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24099 entries, 0 to 24098
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   neighbourhood_cleansed  24099 non-null  float64
 1   property_type           24099 non-null  float64
 2   room_type               24099 non-null  float64
 3   accommodates            24099 non-null  float64
 4   bathrooms               24099 non-null  float64
 5   bedrooms                24099 non-null  float64
 6   beds                    24099 non-null  float64
 7   price                   24099 non-null  float64
dtypes: float64(8)
memory usage: 1.5 MB


In [71]:
# Verificando tipos 'objetos'.

dataset.select_dtypes(include='object').describe()

,neighbourhood_cleansed,property_type,room_type,price
count,39202,39202,39202,24267
unique,224,80,4,914
top,Bedford-Stuyvesant,Entire rental unit,Entire home/apt,$100.00
freq,2760,16036,20853,653


In [86]:
# Verificando tipos 'float'.

dataset.select_dtypes(include='float').describe()

,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms,bedrooms,beds,price
count,2.409900e+04,2.409900e+04,2.409900e+04,2.409900e+04,2.409900e+04,2.409900e+04,2.409900e+04,2.409900e+04
mean,-4.953367e-17,-7.946025e-17,-3.773994e-17,-1.155786e-16,1.238342e-17,2.358746e-17,-8.019736e-17,-1.085023e-16
std,1.000021e+00,1.000021e+00,1.000021e+00,1.000021e+00,1.000021e+00,1.000021e+00,1.000021e+00,1.000021e+00
min,-9.290842e-01,-2.163326e+00,-1.422157e+00,-1.340744e+00,-1.953924e-01,-1.636830e+00,-1.736098e+00,-8.692455e-01
25%,-4.625092e-01,-3.652300e-01,-3.897888e-01,-5.440924e-01,-1.259560e-01,-8.046549e-01,-7.580946e-01,-8.692455e-01
50%,-4.625092e-01,-3.652300e-01,-3.897888e-01,-5.440924e-01,-7.735059e-02,-1.691756e-01,-7.580946e-01,-8.692455e-01
75%,4.706407e-01,-3.652300e-01,6.425796e-01,2.525588e-01,4.981103e-03,8.899566e-01,1.047449e+00,1.112866e+00
max,6.069540e+00,2.570716e+01,1.509574e+01,3.211861e+01,9.898950e+01,1.707001e+00,2.852994e+00,2.103922e+00


In [87]:
# Verificando tipos 'int'.

dataset.select_dtypes(include='int').describe()

ValueError: Cannot describe a DataFrame without columns

In [80]:
# dropando valores NaN.

dataset = dataset.dropna()

# Convertendo tipo obj em string.

dataset[['neighbourhood_cleansed','property_type','room_type',]] = dataset[['neighbourhood_cleansed','property_type','room_type',]].astype('string')

# Convertendo 'price' para float e removendo o $.

dataset['price'] = dataset['price'].replace({'\$': '', ',': ''}, regex=True).astype(float)

# Normalizando as categoricas.

le = LabelEncoder()

# Selecionando as colunas categoricas.

dataset_categoricas = dataset[['neighbourhood_cleansed', 'property_type', 'room_type']]

for col in dataset_categoricas.columns:
    # Atribuindo os valores usando o .loc para modificar o DataFrame original.
    dataset_categoricas.loc[:, col] = le.fit_transform(dataset_categoricas[col])

# Normalizando as numericas.

scaler = StandardScaler()

# Selecionando as colunas numéricas.

dataset_numericas = dataset[['accommodates', 'bathrooms', 'bedrooms', 'beds', 'price']]

dataset = pd.concat([dataset_numericas, dataset_categoricas], axis=1)

# Ajustando o scaler às colunas numéricas.

dataset = scaler.fit_transform(dataset)

<ipython-input-80-6886968ae83e>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_categoricas.loc[:, col] = le.fit_transform(dataset_categoricas[col])
<ipython-input-80-6886968ae83e>:23: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  dataset_categoricas.loc[:, col] = le.fit_transform(dataset_categoricas[col])
<ipython-input-80-6886968ae83e>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [84]:
# Convertendo o array NumPy para DataFrame com os nomes das colunas.

col = ['neighbourhood_cleansed', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'price']

dataset = pd.DataFrame(dataset, columns=col)

In [89]:
X = dataset.drop(columns=['price'])
y = dataset['price']

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=16)

X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [90]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(30, activation='relu', input_shape=X_train.shape[1:]),
  tf.keras.layers.Dropout(0.2),  # Introduce dropout for regularization
  tf.keras.layers.Dense(16, activation='relu'),  # Add an additional hidden layer
  tf.keras.layers.Dense(1, activation='sigmoid')  # Use appropriate activation for binary classification
])

In [91]:
##Criando arquitetura do modelo

model = Sequential()
model.add(Dense(30, activation='relu', input_shape=X_train.shape[1:]))
model.add(Dense(1))

In [92]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 30)                240       
                                                                 
 dense_11 (Dense)            (None, 1)                 31        
                                                                 
Total params: 271 (1.06 KB)
Trainable params: 271 (1.06 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [93]:
##compilando modelo

model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['mean_absolute_error'])

In [94]:
##Treinando modelo (completar)

history = model.fit(X_train, y_train,
                    epochs=100,
                    verbose=1,
                    validation_data=(X_valid, y_valid))

Epoch 1/100
452/452 [==============================] - 1s 2ms/step - loss: 0.3151 - mean_absolute_error: 0.3278 - val_loss: 0.1012 - val_mean_absolute_error: 0.2263
Epoch 2/100
452/452 [==============================] - 1s 2ms/step - loss: 0.2609 - mean_absolute_error: 0.2489 - val_loss: 0.1287 - val_mean_absolute_error: 0.2070
Epoch 3/100
452/452 [==============================] - 1s 2ms/step - loss: 0.2121 - mean_absolute_error: 0.2046 - val_loss: 0.0853 - val_mean_absolute_error: 0.1984
Epoch 4/100
452/452 [==============================] - 1s 2ms/step - loss: 0.0816 - mean_absolute_error: 0.1839 - val_loss: 0.0708 - val_mean_absolute_error: 0.1763
Epoch 5/100
452/452 [==============================] - 1s 2ms/step - loss: 0.0728 - mean_absolute_error: 0.1726 - val_loss: 0.0654 - val_mean_absolute_error: 0.1646
Epoch 6/100
452/452 [==============================] - 1s 2ms/step - loss: 0.0677 - mean_absolute_error: 0.1643 - val_loss: 0.0609 - val_mean_absolute_error: 0.1609
Epoch 7/10

**Avaliando Resultados**

In [95]:
mse_test = model.evaluate(X_test,y_test)

151/151 [==============================] - 0s 2ms/step - loss: 0.0266 - mean_absolute_error: 0.0554


In [96]:
X_new = X_test[:3]

y_pred = model.predict(X_new)

print(y_pred)

1/1 [==============================] - 0s 53ms/step
[[-0.8777269]
 [ 1.1074343]
 [-0.884102 ]]
